In [1]:
#Importing packages
#
# You can delete whatever package(s) you want..
#
import os
import json
import time
import requests
import math as m
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np



# Leave This code here and lookup at pandas documentation 
# if you need to know about chained assignments
pd.options.mode.chained_assignment = None  # default='warn'

# Because most of my data sets I have made can have up to 30+ columns and 20+ rows
# code below will increase pandas defaults for max rows 
# and columns that you can display in a Juptyer Notebook
pd.options.display.max_columns = 60
pd.options.display.max_rows = 90

## Get Pool_IDs for all pools

In [2]:
# Get our block frost api key from the file
with open('keys.txt', 'r') as file:
    api_key = file.read().replace('\n', '')
api_key = api_key.split('=')[1]

In [3]:
# We need a loop to go and fetch all the pool_ids pages and it will end once the length pool_lst is 0
pools_df = pd.DataFrame()
number_of_pools = 5000
headers = {'project_id': '{key}'.format(key=api_key)}

# Loop has to start at 1 because the blockfrost API `pages` starts at 1
for i in range(1, number_of_pools):
	url_pool = 'https://cardano-mainnet.blockfrost.io/api/v0/pools?page='+str(i)
	pool_lst = requests.get(url_pool, headers=headers).json()
	if len(pool_lst) == 0:
		break
	pool_lst = pd.DataFrame(pool_lst)
	pools_df = pools_df.append(pool_lst)
	time.sleep(1)


In [4]:
pools_df.rename(columns={0: 'pool_id'}, inplace=True)
pool_lst = list(pools_df['pool_id'])

In [7]:
pool_lst 

['...',
 ...]

In [5]:
none_relays = {'dns':'', 'dns_srv':'', 'ipv4':'', 'ipv6':'', 'port':''}


## This is going to be the final modified loop :) 

In [8]:
relay_id = pd.DataFrame()
pool_id_df = pd.DataFrame()
meta_df = pd.DataFrame()
stake_df = pd.DataFrame()
headers = {'project_id': '{key}'.format(key=api_key)}



for t in range(len(pools_df)):
    # Our data request from Blockfrost API and store it in a dataframe
    relay_info = 'https://cardano-mainnet.blockfrost.io/api/v0/pools/'+pool_lst[t]+'/relays'
    meta = 'https://cardano-mainnet.blockfrost.io/api/v0/pools/'+pool_lst[t]+'/metadata'
    live = 'https://cardano-mainnet.blockfrost.io/api/v0/pools/'+pool_lst[t]
    req_relays = requests.get(relay_info, headers=headers).json()
    time.sleep(1)  
    req_meta = requests.get(meta, headers=headers).json()
    time.sleep(1)  
    req_live = requests.get(live, headers=headers).json()

    delegs = req_live['live_delegators']
    pool = pool_lst[t]
    stake = req_live['live_stake']
    # Break the loop if there is no data
    if len(req_relays) == 0:
        relay_id = relay_id.append(pd.Series(none_relays), ignore_index=True)
        pool_id_df = pool_id_df.append(pd.Series(delegs), ignore_index=True)
        meta_df = meta_df.append(pd.Series(req_meta), ignore_index=True)
        stake_df = stake_df.append(pd.Series(stake), ignore_index=True)
        print('No relays found')
    
    

    if len(req_relays) > 0:
        i = 0
        while i < len(req_relays):
            relay_id = relay_id.append(req_relays[i], ignore_index=True)
            pool_id_df = pool_id_df.append(pd.Series(delegs), ignore_index=True)
            meta_df = meta_df.append(pd.Series(req_meta), ignore_index=True)
            stake_df = stake_df.append(pd.Series(stake), ignore_index=True)
            i += 1
    # Sleep 1 second to avoid hitting the rate limit
    time.sleep(1)

No relays found
No relays found
No relays found
No relays found
No relays found
No relays found
No relays found
No relays found
No relays found
No relays found
No relays found
No relays found
No relays found
No relays found
No relays found
No relays found
No relays found
No relays found
No relays found
No relays found
No relays found
No relays found
No relays found
No relays found
No relays found
No relays found
No relays found
No relays found
No relays found


In [9]:
pool_id_df.rename(columns={0: 'delegators'}, inplace=True)
stake_df.rename(columns={0: 'live_stake'}, inplace=True)
result = pd.concat([relay_id, stake_df, pool_id_df, meta_df ], axis=1)
stake_lst = result['live_stake'].tolist()
stake_lst = [int(i) for i in stake_lst]
stake_lst = pd.Series(stake_lst)
# result['stake_million_ada'] = stake_lst/1000000

result

,dns,dns_srv,ipv4,ipv6,port,live_stake,delegators,description,hash,hex,homepage,name,pool_id,ticker,url
0,None,None,54.220.20.40,None,3002,47810030816650,3369.0,Octa's Performance Pool,ca7d12decf886e31f5226b5946c62edc81a7e40af95ce7...,153806dbcd134ddee69a8c5204e38ac80448f62342f8c2...,https://octaluso.dyndns.org,OctasPool,pool1z5uqdk7dzdxaae5633fqfcu2eqzy3a3rgtuvy087f...,OCTAS,https://raw.githubusercontent.com/Octalus/card...
1,relays.mainnet.stakenuts.com,None,None,None,3001,19187423160709,165.0,StakeNuts.com,47c0c68cb57f4a5b4a87bad896fc274678e7aea98e200f...,0f292fcaa02b8b2f9b3c8f9fd8e0bb21abedb692a6d505...,https://stakenuts.com/,StakeNuts,pool1pu5jlj4q9w9jlxeu370a3c9myx47md5j5m2str0na...,NUTS,https://stakenuts.com/mainnet.json
2,relay.zenithpool.io,None,None,None,31400,550903867804,10.0,"One owner, one operator, one pool. Operating s...",21cc44498ab6e74e081a805e629fff171a66d0c8d034f9...,c1ede3cc9133209466774d4826044e408db13d6fe6df75...,https://zenithpool.io,ZENITH,pool1c8k78ny3xvsfgenhf4yzvpzwgzxmz0t0um0h2xnn2...,ZEN,https://metadata.zenithpool.io
3,None,None,95.183.53.20,None,6060,1810331128598,701.0,"Stability, Security, Reliability, Neutrality! ...",34fdde237812fab14d29a80423bb295f39122f4fea1aae...,01df29429173d263c7533a22742dae19f16a08798b7a57...,https://ispool.live/,Switzerland Investment,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,000,https://ispool.live/metadata
4,None,None,95.217.222.194,None,6060,1810331128598,701.0,"Stability, Security, Reliability, Neutrality! ...",34fdde237812fab14d29a80423bb295f39122f4fea1aae...,01df29429173d263c7533a22742dae19f16a08798b7a57...,https://ispool.live/,Switzerland Investment,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...,000,https://ispool.live/metadata
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5457,relay2.kyotopool.net,None,None,None,3000,17620730,1.0,Hold for City of Love,5f861741484e8395a76f74b9df52e89246e4d335bff706...,2d13883a146158e4aa46aaec59b7e6e45602bfebeae1a0...,https://Kyotopool.net/,KYOTO Pool,pool195fcsws5v9vwf2jx4tk9ndlxu3tq90ltats6qfwnw...,KYOTO,https://git.io/JPcFB
5458,relay1.nilepool.net,None,None,None,3000,17426497,1.0,Hold for River of Love,32a5cea6c877a9593203236f178c289dd18f3e794ffeb3...,353bb977d972b79852225f5258f39c674884d95da27495...,https://nilepool.net/,NILE Pool,pool1x5amja7ew2mes53ztaf93uuuvaygfk2a5f6f2jncg...,NILE,https://git.io/JPRG3
5459,relay2.nilepool.net,None,None,None,3000,17426497,1.0,Hold for River of Love,32a5cea6c877a9593203236f178c289dd18f3e794ffeb3...,353bb977d972b79852225f5258f39c674884d95da27495...,https://nilepool.net/,NILE Pool,pool1x5amja7ew2mes53ztaf93uuuvaygfk2a5f6f2jncg...,NILE,https://git.io/JPRG3
5460,None,None,85.214.115.3,None,7766,236578277,2.0,"changemakers, innovators, visionaries - let´s ...",4e5c87056491e6f88c7a30e945ff897c7401f381661fc1...,f685ec1bd556b584dc764b9c2aca83d65d4231268d4233...,https://krypto-bergbau.de,BR Krypto-Bergbau,pool176z7cx74266cfhrkfwwz4j5r6ew5yvfx34prxldaq...,BRKB,https://raw.githubusercontent.com/Bensen90/met...


In [10]:
result.to_excel(r'relays.xlsx', index = False)
result.to_json(r'relays.json', orient='records')
result.to_csv(r'relays.csv', index = False)